# 在PySpark中实现你自己的推荐系统
现今，推荐系统被用来个性化你在网上的体验，告诉你买什么，去哪里吃，甚至是你应该和谁做朋友。人们口味各异，但通常有迹可循。人们倾向于喜欢那些与他们所喜欢的东西类似的东西，并且他们倾向于与那些亲近的人有相似的口味。推荐系统试图捕捉这些模式，以助于预测你还会喜欢什么东西。电子商务、社交媒体、视频和在线新闻平台已经积极的部署了它们自己的推荐系统，以帮助它们的客户更有效的选择产品，从而实现双赢。

两种最普遍的推荐系统的类型是基于内容和协同过滤（CF）。协同过滤基于用户对产品的态度产生推荐，也就是说，它使用“人群的智慧”来推荐产品。与此相反，基于内容的推荐系统集中于物品的属性，并基于它们之间的相似性为你推荐。一般情况下，协作过滤（CF）是推荐引擎的主力。该算法具有能够自身进行特征学习的一个非常有趣的特性，这意味着它可以开始学习使用哪些特性。CF可以分为基于内存的协同过滤和基于模型的协同过滤。在本教程中，你将使用奇异值分解（SVD）实现基于模型的CF和通过计算余弦相似实现基于内存的CF。
我们将使用MovieLens数据集，它是在实现和测试推荐引擎时所使用的最常见的数据集之一。它包含来自于943个用户以及精选的1682部电影的100K个电影打分。你应该添加解压缩的movielens数据文件夹你的notebook目录下。你也可以在[这里下载数据集](http://files.grouplens.org/datasets/movielens/ml-100k.zip)。

In [1]:
!ls

'7.2 pyspark.ipynb'  '8.2 Recommendation Engine.ipynb'	 data   未命名.ipynb


In [2]:
sc.master

'local[*]'

In [3]:
%%time
f = open('data/u.data')
len(f.readlines())

CPU times: user 12.6 ms, sys: 48 µs, total: 12.6 ms
Wall time: 22.9 ms


### real time
是从进行开始执行到完成所经历的墙上时钟时间（wall clock）时间，包括其他进程使用的时间片（time slice）和本进程耗费在阻塞（如等待I/O操作完成）上的时间。
### user time
是进程执行用户态代码（内核外）耗费的CPU时间，仅统计该进程执行时实际使用的CPU时间，而不计入其他进程使用的时间片和本进程阻塞的时间
### sys time 
是该进程在内核态运行所耗费的CPU时间，即内核执行系统调用所使用的CPU时间

CPU总时间（user + sys）是CPU执行用户进程操作和内核（代表用户进程执行）系统调用所耗时间的总和，即该进程（包括线程和子进程）所使用的实际CPU时间。若程序循环遍历数组，则增加用户CPU时间；若程序执行exec或fork等系统调用，则增加系统CPU时间。
在多核处理器机器上，若进程含有多个线程或通过fork调用创建子进程，则实际时间可能小于CPU总时间，因为不同线程或进程可并行执行，但其时间会计入主进程的CPU总时间。若程序在某段时间处于等待状态而并未执行，则实际时间可能大于CPU总时间：
- real < CPU  表明进程为计算密集型（CPU bound），利用多核处理器的并行执行优势
- real ≈ CPU  表明进程为计算密集型，未并行执行
- real > CPU  表明进程为I/O密集型 （I/O bound），多核并行执行优势并不明显

In [5]:
#rawUserData=sc.textFile("hdfs://master:9000/user/hduser/data/u.data")
rawUserData=sc.textFile("data/u.data")
rawUserData.count()

100000

时钟时间 ＝ 处于阻塞状态时间 ＋ 处于就绪状态时间 ＋处于运行状态时间

In [6]:
rawUserData.first()

'196\t242\t3\t881250949'

In [8]:
from pyspark.mllib.recommendation import Rating
ratingsRDD=rawUserData.map(lambda line:line.split("\t")[:3])
ratingsRDD.take(5)

[['196', '242', '3'],
 ['186', '302', '3'],
 ['22', '377', '1'],
 ['244', '51', '2'],
 ['166', '346', '1']]

In [9]:
# 查看数据的总记录行数
numRating=ratingsRDD.count()
numRating

100000

In [10]:
# 找出一共有多少不同的用户
numUsers = ratingsRDD.map(lambda x:x[0]).distinct().count()
numUsers 

943

In [11]:
# 找出一共有多少部不同的电影
numMovies = ratingsRDD.map(lambda x:x[1]).distinct().count()
numMovies

1682

In [15]:
#==================================
#            构建训练模型
#==================================
# RatingsRDD ===by ALS.train==>>MatirixFactorization Model
from pyspark.mllib.recommendation import ALS
# ALS.train(RDD data,rank(k,to m*k),10(Iterations,5 is defalut value),0.01(lambda 0.01 is default value))
model = ALS.train(ratingsRDD,10,10,0.01) 
print(model)
#model is MatrixFactorizationModel object


In [16]:
%%time
# use model to recommend for user
#model.recommendProducts(userID,recommendation movie numbers)
model.recommendProducts(100,5)

CPU times: user 1.37 ms, sys: 65 µs, total: 1.44 ms
Wall time: 186 ms


[Rating(user=100, product=1589, rating=6.5371048620893735),
 Rating(user=100, product=1167, rating=6.32643579934924),
 Rating(user=100, product=253, rating=6.202221682020556),
 Rating(user=100, product=1160, rating=5.940409622449528),
 Rating(user=100, product=1159, rating=5.859149044252257)]

In [17]:
# look recommendation value
model.predict(100,1311)

3.3738405132736946

In [19]:
model.predict(100,1682)

2.296430115224021

In [20]:
# use model to recommend for movie
#model.recommendProducts(movieID,recommendation user numbers)
model.recommendUsers(200,5)

[Rating(user=695, product=200, rating=6.0103228052718745),
 Rating(user=86, product=200, rating=5.966203528738056),
 Rating(user=55, product=200, rating=5.823422992513373),
 Rating(user=400, product=200, rating=5.638304169647098),
 Rating(user=355, product=200, rating=5.541419478090867)]

In [21]:
# show recommend movie title
itemRDD = sc.textFile("/home/tr/pythonwork/PythonProject/data/u.item")
itemRDD.count()

1682

In [22]:
itemRDD.first()

'1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0'

In [23]:
# creat dict for movie ID and TITLE
movieTitle = itemRDD.map(lambda line:line.split("|")).map(lambda x:(float(x[0]),x[1])).collectAsMap()
len(movieTitle)

1682

In [24]:
sorted(movieTitle.items())[0:5]

[(1.0, 'Toy Story (1995)'),
 (2.0, 'GoldenEye (1995)'),
 (3.0, 'Four Rooms (1995)'),
 (4.0, 'Get Shorty (1995)'),
 (5.0, 'Copycat (1995)')]

In [25]:
movieTitle[5]

'Copycat (1995)'

In [26]:
%%time
#==================================
#            预测模型 
#==================================
# 显示前5个推荐的电影 
recommendP = model.recommendProducts(100,5)
for p in recommendP:
    print("USER ID:  " + str(p[0]) + "  ,Recommend Movie:  " + str(movieTitle[p[1]]) + "  ,Rating:" + str(p[2]))

USER ID:  100  ,Recommend Movie:  Schizopolis (1996)  ,Rating:6.5371048620893735
USER ID:  100  ,Recommend Movie:  Sum of Us, The (1994)  ,Rating:6.32643579934924
USER ID:  100  ,Recommend Movie:  Pillow Book, The (1995)  ,Rating:6.202221682020556
USER ID:  100  ,Recommend Movie:  Love! Valour! Compassion! (1997)  ,Rating:5.940409622449528
USER ID:  100  ,Recommend Movie:  Stalker (1979)  ,Rating:5.859149044252257
CPU times: user 1.49 ms, sys: 0 ns, total: 1.49 ms
Wall time: 153 ms


## 直接使用python构建推荐模型

In [27]:
#查看python的安装路径
! which python
! which pip

/home/tr/anaconda3/bin/python
/home/tr/anaconda3/bin/pip


In [28]:
import pandas as pd
import numpy as np
import tensorflow as tf
print(pd.__version__)
print(np.__version__)
print(tf.__version__)

0.24.2
1.16.2
1.13.1


In [29]:
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('data/u.data', sep='\t', names=header)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [30]:
# 计算唯一用户和电影的数量
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items)  )

Number of users = 943 | Number of movies = 1682


In [34]:
from sklearn import model_selection as cv
# 来将数据混洗并分割成两个数据集:训练集和验证集
train_data, test_data = cv.train_test_split(df, test_size=0.25)
print(train_data.shape)
print(test_data.shape)

(75000, 4)
(25000, 4)


## 基于内存的协同过滤
基于内存的协同过滤方法可以分为两个主要部分：用户-产品协同过滤和产品-产品协同过滤。一个用户-产品协同过滤将选取一个特定的用户，基于打分的相似性发现类似于该用户的用户，并推荐那些相似用户喜欢的产品。相比之下，产品-产品协同过滤会选取一个产品，发现喜欢该产品的用户，并找到这些用户或相似的用户还喜欢的其他的产品。输入一个产品，然后输出其他产品作为推荐。

- 用户-产品协同过滤: “喜欢这个东西的人也喜欢……”
- 产品-产品协同过滤: “像你一样的人也喜欢……”

在这两种情况下，从整个数据集构建一个用户-产品矩阵。由于你已经将数据拆分到测试集和训练集，那么你将需要创建两个[943 x 1682]矩阵。训练矩阵包含75%的打分，而测试矩阵包含25%的打分。

通常用于推荐系统中的距离矩阵是余弦相似性，其中，打分被看成n维空间中的向量，而相似性是基于这些向量之间的角度进行计算的。用户a和m的余弦相似性可以使用下面的公式进行计算，其中，获取用户向量的点积$u_k$和$u_a$，然后用向量的欧几里得长度的乘积来除以它。

$s_{u}^{\cos }\left(u_{k}, u_{a}\right)=\frac{u_{k} \cdot u_{a}}{\left\|u_{k}\right\|\left\|u_{a}\right\|}=\frac{\sum x_{k, m} x_{a, m}}{\sqrt{\sum x_{k, m}^{2} \sum x_{a, m}^{2}}}$

要计算产品m和b之间的相似性，使用公式：

$s_{u}^{\cos }\left(i_{m}, i_{b}\right)=\frac{i_{m} \cdot i_{b}}{\left\|i_{m}\right\|\left\|i_{b}\right\|}=\frac{\sum x_{a, m} x_{a, b}}{\sqrt{\sum x_{a, m}^{2} \sum x_{a, b}^{2}}}$

#### 第一步是创建用户-产品矩阵。由于你既有测试数据，又有训练数据，那么你需要创建两个矩阵。

In [35]:
# 创建(n_users,n_items)的矩阵
train_data_matrix = np.zeros((n_users, n_items))
train_data_matrix.shape

(943, 1682)

In [1]:
# 这部分影响整体性能，不再展示
#for row in train_data.itertuples():
#    print(row)
#得到结果
#Pandas(Index=69103, user_id=522, item_id=530, rating=4, timestamp=876961314)
#Pandas(Index=25688, user_id=347, item_id=240, rating=5, timestamp=881653300)

In [38]:
# itertuples 将 DataFrame迭代为元祖
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  

In [39]:
test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [40]:
%%time
train_data_matrix.shape

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


(943, 1682)

In [41]:
# 可以使用sklearn的pairwise_distances函数来计算余弦相似性。注意，输出范围从0到1，因为打分都是正的。
# 余弦相似度用向量空间中两个向量夹角的余弦值作为衡量两个个体间差异的大小。
# 余弦值越接近1，就表明夹角越接近0度，也就是两个向量越相似，这就叫"余弦相似性"。

from sklearn.metrics.pairwise import pairwise_distances

user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

已经创建了相似性矩阵：user_similarity和item_similarity，因此，下面就可以通过为基于用户的CF应用下面的公式做出预测：

$\hat{x}_{k, m}=\overline{x}_{k}+\frac{\sum_{u_{a}} \operatorname{sim}_{u}\left(u_{k}, u_{a}\right)\left(x_{a, m}-x_{u_{a}}^{-}\right)}{\sum_{u_{a}}\left|\operatorname{sim}_{u}\left(u_{k}, u_{a}\right)\right|}$

你可以将用户k和a之间的相似性看成权重，它乘以相似用户a (校正的平均评分用户)的评分。你需要规范化该值，使打分位于1到5之间，最后，对你尝试预测的用户的平均评分求和。

这里的想法是，某些用户可能会倾向于对所有的电影，总是给予高或低评分。这些用户提供的评分的相对差比绝对评分值更重要。举个例子：假设，用户k对他最喜欢的电影打4星，而对所有其他的好电影打3星。现在假设另一个用户t对他/她喜欢的电影打5星，而对他/她感到无聊的电影打3星。那么这两个用户可能品味非常相似，但对打分系统区别对待。

当为基于产品的CF进行预测时，你无须纠正用户的平均打分，因为查询用户本事就是用来做预测的。

$\hat{x}_{k, m}=\frac{\sum_{i_{b}} \operatorname{sim}_{i}\left(i_{m}, i_{b}\right)\left(x_{k, b}\right)}{\sum_{i_{b}}\left|\operatorname{sim}_{i}\left(i_{m}, i_{b}\right)\right|}$

In [42]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.heatmap(item_similarity)

In [43]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [84]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

# 评估
有许多评价指标，但其中最受欢迎的用来度量预测评分的准确性的指标是均方根误差 (RMSE)。反映一个数据集的离散程度。

$R M S E=\sqrt{\frac{1}{N} \sum\left(x_{i}-\hat{x}_{i}\right)^{2}}$

你可以使用sklearn的mean_square_error (MSE)函数，其中，RMSE仅仅是MSE的平方根。要了解更多不同的评价指标，你可以看看[这篇文章](http://research.microsoft.com/pubs/115396/EvaluationMetrics.TR.pdf)。

由于你只是想要考虑测试数据集中的预测评分，因此，使用prediction[ground_truth.nonzero()]筛选出预测矩阵中的所有其他元素。

In [85]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [86]:
print ('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print ('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.1170261362946023
Item-based CF RMSE: 3.444137570041472


In [87]:
sparsity=round(1.0-len(df)/float(n_users*n_items),3)
print ('MovieLens100K 稀疏度: ' +  str(sparsity*100) + '%')

MovieLens100K 稀疏度: 93.7%


基于内存的算法事很容易实现并产生合理的预测质量的。

基于内存的CF的缺点是，它不能扩展到真实世界的场景，并且没有解决众所周知的冷启动问题，也就是当新用户或新产品进入系统时。基于模型的CF方法是可扩展的，并且可以比基于内存的模型处理更高的稀疏度，但当没有任何评分的用户或产品进入系统时，也是苦不堪言的。

In [88]:
user_prediction.shape

(943, 1682)

In [92]:
# userid 为 120 的用户 
userID = 120
print(np.argmax(user_prediction[userID]))
print(user_prediction[userID,np.argmax(user_prediction[userID])])

49
1.943450073264475


In [108]:
movie_df = pd.read_table('data/u.item',sep='|', encoding='ISO-8859-1',header=None)
movie_df.head()

/home/tr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [118]:
movie_df.iloc[userID,1]

'Independence Day (ID4) (1996)'

In [121]:
# 预测存在的问题
print(user_prediction.min(),user_prediction.max())
print(test_data_matrix.min(),test_data_matrix.max())

-0.15751835709904244 2.7573858340292583
0.0 5.0


和真实值差的较大，有没有其他方法能降低RMSE的值

## 作业：
仍然使用该数据集，请使用tensorflow或者pytorch在notebook中完成该项目，并上传到云班课。使用tensorboard查看loss曲线
并能得到如下的类似结果

您要想哪位用户进行推荐？请输入用户编号：123

- ==============================为该用户推荐的评分最高的10部电影是：===============================
- 评分: 5.03, 电影名: Fireworks Wednesday (Chaharshanbe-soori) (2006)
- 评分: 4.88, 电影名: Woman on the Beach (Haebyeonui yeoin) (2006)
- 评分: 4.73, 电影名: Mummy's Ghost, The (1944)
- 评分: 4.66, 电影名: Maborosi (Maboroshi no hikari) (1995)
- 评分: 4.63, 电影名: Boiling Point (1993)
- 评分: 4.60, 电影名: Mala Noche (1985)
- 评分: 4.49, 电影名: All-Star Superman (2011)
- 评分: 4.47, 电影名: Bill Hicks: Relentless (1992)
- 评分: 4.45, 电影名: Something Borrowed (2011)
- 评分: 4.37, 电影名: Box of Moon Light (1996)
